# Lab 1 - Module 5: Mountain Landscape Search

**Learning Objectives:**
- Search in 2D space for optimal point
- Understand local vs. global maxima
- Connect to ML loss landscapes

**Time:** ~15-20 minutes

---

**IMPORTANT:** Enter the same group code!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import FloatSlider, Button, Output, HBox, VBox
from IPython.display import display

group_code = int(input("Enter your group code: "))
np.random.seed(group_code)

# Generate mountain landscape
num_peaks = np.random.randint(3, 6)
peak_centers = []
peak_heights = []
peak_widths = []

for _ in range(num_peaks):
    cx = np.random.uniform(-3.0, 3.0)
    cy = np.random.uniform(-3.0, 3.0)
    height = np.random.uniform(1.0, 5.0)
    width = np.random.uniform(0.6, 1.5)
    peak_centers.append((cx, cy))
    peak_heights.append(height)
    peak_widths.append(width)

def mountain_height(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.zeros_like(x, dtype=float)
    for (cx, cy), h, w in zip(peak_centers, peak_heights, peak_widths):
        z += h * np.exp(-(((x - cx)**2 + (y - cy)**2) / (2 * w**2)))
    return z

print(f"✓ Mountain landscape created with {num_peaks} peaks!")

✓ Mountain landscape created with 4 peaks!


## 5. Finding a Mountain Peak – Optimization Beyond Line Fitting

So far, our optimization examples involved **fitting a line** to data:

- Parameters: slope *m* and intercept *b*.
- Goal: choose (*m*, *b*) to make the **global error** (loss) small.

But optimization and learning are **not just about line fitting**.

In many problems, we want to:

- **maximize** something (e.g., profit, altitude, reward), or  
- **minimize** something (e.g., cost, time, error),

over a space of possible choices.

In this activity, imagine you are exploring a **mountain range**:

- For any coordinate *(x, y)*, there is an **altitude** `h(x, y)`.
- You can “visit” points in the landscape and measure the altitude, but:
  - you **cannot** see the whole mountain range,
  - you only see the locations you choose and their altitudes.

However, the landscape is tricky:

- There is **not just one peak**, but **several peaks**.
- Some peaks are higher than others.

Your job is to use **samples of altitude** to:

- search for high regions,
- try to find the **highest peak** you can,
- and notice the difference between **local peaks** and a **global peak**.

At the end, we’ll reveal the full mountain landscape and compare:

- your best-found peak,
- the true global peak,
- and the other local peaks you might have visited.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import FloatSlider, Button, Output, HBox, VBox
from IPython.display import display

# ------------------------------------------------------------
# 1. Define the hidden mountain landscape (multiple peaks)
# ------------------------------------------------------------

# Assume np.random.seed(group_code) has already been called earlier.
# We generate a sum of Gaussian bumps to create several peaks.

X_MIN, X_MAX = -5.0, 5.0
Y_MIN, Y_MAX = -5.0, 5.0

num_peaks = np.random.randint(3, 6)  # between 3 and 5 peaks

peak_centers = []
peak_heights = []
peak_widths = []

for _ in range(num_peaks):
    cx = np.random.uniform(-3.0, 3.0)
    cy = np.random.uniform(-3.0, 3.0)
    height = np.random.uniform(1.0, 5.0)
    width = np.random.uniform(0.6, 1.5)

    peak_centers.append((cx, cy))
    peak_heights.append(height)
    peak_widths.append(width)

def mountain_height(x, y):
    """Hidden mountain landscape: sum of Gaussian peaks."""
    x = np.asarray(x)
    y = np.asarray(y)
    z = np.zeros_like(x, dtype=float)
    for (cx, cy), h, w in zip(peak_centers, peak_heights, peak_widths):
        z += h * np.exp(-(((x - cx)**2 + (y - cy)**2) / (2 * w**2)))
    return z

print(f"Hidden mountain landscape created with {num_peaks} peaks (kept secret from students).")

# ------------------------------------------------------------
# 2. Widgets and state for interactive sampling
# ------------------------------------------------------------

x_slider_2d = FloatSlider(
    description="x",
    min=X_MIN,
    max=X_MAX,
    step=0.2,
    value=0.0,
    continuous_update=False
)

y_slider_2d = FloatSlider(
    description="y",
    min=Y_MIN,
    max=Y_MAX,
    step=0.2,
    value=0.0,
    continuous_update=False
)

sample_button = Button(
    description="Measure altitude",
    button_style="info"
)

done_button_2d = Button(
    description="Done",
    button_style="success"
)

out_2d_game = Output()
out_2d_reveal = Output()

# Store samples as a list of dicts: {attempt, x, y, height}
samples_2d = []

def on_sample_clicked(b_widget):
    """Record a sample (x, y, height) and update the plot + table."""
    x_guess = x_slider_2d.value
    y_guess = y_slider_2d.value
    h_val = float(mountain_height(x_guess, y_guess))

    samples_2d.append({
        "attempt": len(samples_2d) + 1,
        "x": x_guess,
        "y": y_guess,
        "height": h_val
    })

    df = pd.DataFrame(samples_2d)

    with out_2d_game:
        out_2d_game.clear_output(wait=True)
        print("Mountain optimization game:")
        print(f"- Choose (x, y) within [{X_MIN}, {X_MAX}] x [{Y_MIN}, {Y_MAX}].")
        print("- Click 'Measure altitude' to see the height at that point.")
        print("- You only see the sampled points, not the whole mountain.\n")

        print("Recent samples:")
        display(df.tail(10))

        # Plot sampled points in (x, y), colored by altitude
        plt.figure(figsize=(7, 5))

        xs = df["x"].values
        ys = df["y"].values
        hs = df["height"].values

        sc = plt.scatter(xs, ys, c=hs, cmap="plasma", s=80, edgecolor="black")
        plt.xlabel("x")
        plt.ylabel("y")
        plt.xlim(X_MIN, X_MAX)
        plt.ylim(Y_MIN, Y_MAX)
        plt.title("Your sampled locations (color = altitude)")
        cbar = plt.colorbar(sc)
        cbar.set_label("Altitude")
        plt.grid(True)
        plt.show()

        # Highlight the current best sample
        best_idx = df["height"].idxmax()
        best_row = df.loc[best_idx]
        print(f"Current best sample: attempt {int(best_row['attempt'])}, "
              f"x = {best_row['x']:.2f}, y = {best_row['y']:.2f}, "
              f"height = {best_row['height']:.3f}")

def on_done_clicked_2d(b_widget):
    """Reveal the full landscape, global peak, and the group's best sample."""
    sample_button.disabled = True
    done_button_2d.disabled = True
    x_slider_2d.disabled = True
    y_slider_2d.disabled = True

    if not samples_2d:
        with out_2d_reveal:
            out_2d_reveal.clear_output()
            print("No samples were taken. Nothing to reveal.")
        return

    # Create a grid to visualize the full landscape
    grid_size = 100
    x_vals = np.linspace(X_MIN, X_MAX, grid_size)
    y_vals = np.linspace(Y_MIN, Y_MAX, grid_size)
    Xg, Yg = np.meshgrid(x_vals, y_vals)
    Zg = mountain_height(Xg, Yg)

    # Find approximate global maximum on grid
    flat_index = np.argmax(Zg)
    i_max, j_max = np.unravel_index(flat_index, Zg.shape)
    x_global = Xg[i_max, j_max]
    y_global = Yg[i_max, j_max]
    h_global = Zg[i_max, j_max]

    df = pd.DataFrame(samples_2d)
    best_idx = df["height"].idxmax()
    best_row = df.loc[best_idx]

    with out_2d_reveal:
        out_2d_reveal.clear_output()
        print("Revealing the mountain landscape, your samples, and the true global peak:\n")

        plt.figure(figsize=(8, 6))
        cs = plt.contourf(Xg, Yg, Zg, levels=30, cmap="plasma")
        cbar = plt.colorbar(cs)
        cbar.set_label("Altitude")

        # Overlay your samples
        sc2 = plt.scatter(df["x"], df["y"],
                          c=df["height"], cmap="plasma",
                          s=80, edgecolor="black",
                          label="Your samples")

        # Highlight your best sample
        plt.scatter([best_row["x"]], [best_row["y"]],
                    color="cyan", marker="o", s=150,
                    edgecolor="black",
                    label="Your best sample")

        # Mark the global maximum on the grid
        plt.scatter([x_global], [y_global],
                    color="red", marker="*",
                    s=200, edgecolor="black",
                    label="Global peak (grid)")

        plt.xlabel("x")
        plt.ylabel("y")
        plt.xlim(X_MIN, X_MAX)
        plt.ylim(Y_MIN, Y_MAX)
        plt.title("Mountain Landscape with Your Samples and the Global Peak")
        plt.legend()
        plt.grid(True)
        plt.show()

        print(f"Your best sample: x = {best_row['x']:.3f}, "
              f"y = {best_row['y']:.3f}, height = {best_row['height']:.3f}")
        print(f"True global peak (grid): x ≈ {x_global:.3f}, "
              f"y ≈ {y_global:.3f}, height ≈ {h_global:.3f}")
        print("\nNotice how many peaks exist and where your exploration focused.")

sample_button.on_click(on_sample_clicked)
done_button_2d.on_click(on_done_clicked_2d)

print("Mountain optimization game:")
print(f"- x and y range: [{X_MIN}, {X_MAX}] x [{Y_MIN}, {Y_MAX}].")
print("- Move the sliders to choose (x, y) and click 'Measure altitude'.")
print("- When you are finished exploring, click 'Done' to reveal the full landscape.\n")

display(VBox([
    HBox([x_slider_2d, y_slider_2d]),
    HBox([sample_button, done_button_2d]),
    out_2d_game,
    out_2d_reveal
]))


Hidden mountain landscape created with 4 peaks (kept secret from students).
Mountain optimization game:
- x and y range: [-5.0, 5.0] x [-5.0, 5.0].
- Move the sliders to choose (x, y) and click 'Measure altitude'.
- When you are finished exploring, click 'Done' to reveal the full landscape.



In [3]:
# Summary
if samples_2d:
    print(f"Total samples: {len(samples_2d)}")
    best_idx = max(range(len(samples_2d)), key=lambda i: samples_2d[i]['height'])
    print(f"Best height: {samples_2d[best_idx]['height']:.4f}")
    print(f"Best (x, y): ({samples_2d[best_idx]['x']:.2f}, {samples_2d[best_idx]['y']:.2f})")
else:
    print("No samples taken yet!")

Total samples: 16
Best height: 7.6080
Best (x, y): (2.40, 1.60)


## Next Steps

1. **Return to the LMS**
2. **Answer Questions 12-15** about the mountain search
3. **Submit your lab** - You're done!

Great work! 🎉